In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/fake news train.csv')
test=pd.read_csv('/content/drive/MyDrive/fake news test.csv')

In [ ]:
train.dropna(inplace=True)

In [ ]:
X=train.drop(['label'], axis=1)
Y=train['label']

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
## vocab size\
voc_size=10000

In [ ]:
messages=X.copy()
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
## data preprocessing 
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
lm = WordNetLemmatizer()
corpus=[]
for i in range(0,len(messages)):
  print(i)
  review=re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[lm.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [ ]:
onehot_rep=[one_hot(words,voc_size)for words in corpus]
print(onehot_rep)

[[6151, 5318, 9081, 97, 5353, 7876, 9551, 6436, 1287, 2667], [2252, 9394, 7730, 7012, 8600, 2677, 4356], [9233, 545, 7956, 4084], [6332, 7612, 389, 9493, 6913, 5877], [5159, 8600, 5022, 9586, 131, 6577, 8600, 3592, 8646, 2878], [5891, 6078, 9553, 5383, 9353, 1297, 88, 564, 2175, 7511, 3374, 6525, 7076, 1118, 4356], [2725, 8723, 2247, 5907, 9529, 209, 3978, 9174, 4970, 1100, 1929], [5734, 2911, 4869, 5927, 9487, 2914, 1297, 7592, 4970, 1100, 1929], [4675, 6611, 8965, 7643, 5671, 3894, 4099, 770, 1297, 5049], [1403, 3700, 113, 3279, 4452, 610, 9618, 3578], [9552, 6288, 2243, 3998, 9340, 9002, 5564, 9067, 4404, 319, 8063], [9493, 7095, 5353, 7774, 1297, 9487], [3937, 3614, 5514, 1681, 4221, 6289, 6109, 2727, 1741], [1359, 9935, 3090, 1289, 530, 4990, 9474, 4970, 1100, 1929], [438, 5133, 3225, 3814, 3582, 4970, 1100, 1929], [3664, 57, 4664, 2857, 2551, 9755, 6122, 8496, 7556, 6299], [2453, 3712, 9394], [5210, 1457, 2555, 2343, 1297, 3506, 9629, 4356], [6632, 1066, 7730, 9013, 9588, 5445, 6

In [ ]:
## fixing the sentence length
sent_lenght=20
embedded_doc=pad_sequences(onehot_rep, padding='pre', maxlen=sent_lenght)

In [ ]:
embedded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 6151,
       5318, 9081,   97, 5353, 7876, 9551, 6436, 1287, 2667], dtype=int32)

In [ ]:
## creating the model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_lenght))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X_final=np.array(embedded_doc)
Y_final=np.array(Y)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X_final,Y_final, test_size=0.2, random_state=44)

In [ ]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 10s 11ms/step - loss: 0.4586 - accuracy: 0.7639 - val_loss: 0.1933 - val_accuracy: 0.9180
Epoch 2/10
229/229 [==============================] - 2s 8ms/step - loss: 0.1209 - accuracy: 0.9526 - val_loss: 0.2033 - val_accuracy: 0.9180
Epoch 3/10
229/229 [==============================] - 2s 8ms/step - loss: 0.0719 - accuracy: 0.9745 - val_loss: 0.2378 - val_accuracy: 0.9095
Epoch 4/10
229/229 [==============================] - 2s 8ms/step - loss: 0.0397 - accuracy: 0.9862 - val_loss: 0.2753 - val_accuracy: 0.9021
Epoch 5/10
229/229 [==============================] - 2s 8ms/step - loss: 0.0257 - accuracy: 0.9920 - val_loss: 0.3634 - val_accuracy: 0.9109
Epoch 6/10
229/229 [==============================] - 2s 8ms/step - loss: 0.0114 - accuracy: 0.9970 - val_loss: 0.4643 - val_accuracy: 0.9109
Epoch 7/10
229/229 [==============================] - 2s 8ms/step - loss: 0.0060 - accuracy: 0.9984 - val_loss: 0.4087 - val_accuracy: 0.9106
Epoc

In [ ]:
## model with dropout layer
## creating the model
from tensorflow.keras.layers import Dropout
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_lenght))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest), epochs=50, batch_size=64)

Epoch 1/50
229/229 [==============================] - 4s 10ms/step - loss: 0.4638 - accuracy: 0.7461 - val_loss: 0.2129 - val_accuracy: 0.9155
Epoch 2/50
229/229 [==============================] - 2s 8ms/step - loss: 0.1164 - accuracy: 0.9539 - val_loss: 0.1965 - val_accuracy: 0.9204
Epoch 3/50
229/229 [==============================] - 2s 8ms/step - loss: 0.0816 - accuracy: 0.9719 - val_loss: 0.2332 - val_accuracy: 0.9109
Epoch 4/50
229/229 [==============================] - 2s 8ms/step - loss: 0.0471 - accuracy: 0.9848 - val_loss: 0.2914 - val_accuracy: 0.9103
Epoch 5/50
229/229 [==============================] - 2s 8ms/step - loss: 0.0349 - accuracy: 0.9901 - val_loss: 0.3097 - val_accuracy: 0.9125
Epoch 6/50
229/229 [==============================] - 2s 8ms/step - loss: 0.0219 - accuracy: 0.9931 - val_loss: 0.4643 - val_accuracy: 0.9065
Epoch 7/50
229/229 [==============================] - 2s 8ms/step - loss: 0.0154 - accuracy: 0.9959 - val_loss: 0.4384 - val_accuracy: 0.9128
Epoch

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
## using bidirectional LSTM
from tensorflow.keras.layers import Bidirectional
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length=sent_lenght))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model1.fit(xtrain,ytrain,validation_data=(xtest,ytest), epochs=50, batch_size=64)

Epoch 1/50
229/229 [==============================] - 5s 13ms/step - loss: 0.4171 - accuracy: 0.7738 - val_loss: 0.2074 - val_accuracy: 0.9193
Epoch 2/50
229/229 [==============================] - 2s 11ms/step - loss: 0.1160 - accuracy: 0.9541 - val_loss: 0.2023 - val_accuracy: 0.9191
Epoch 3/50
229/229 [==============================] - 2s 11ms/step - loss: 0.0615 - accuracy: 0.9779 - val_loss: 0.2261 - val_accuracy: 0.9158
Epoch 4/50
229/229 [==============================] - 2s 11ms/step - loss: 0.0433 - accuracy: 0.9868 - val_loss: 0.3168 - val_accuracy: 0.9084
Epoch 5/50
229/229 [==============================] - 2s 10ms/step - loss: 0.0250 - accuracy: 0.9923 - val_loss: 0.3810 - val_accuracy: 0.9098
Epoch 6/50
229/229 [==============================] - 2s 10ms/step - loss: 0.0167 - accuracy: 0.9949 - val_loss: 0.4366 - val_accuracy: 0.9024
Epoch 7/50
229/229 [==============================] - 2s 10ms/step - loss: 0.0141 - accuracy: 0.9949 - val_loss: 0.4414 - val_accuracy: 0.9068

In [ ]:
ypredict=model1.predict_classes(xtest)
from sklearn.metrics import confusion_matrix
CM=confusion_matrix(ytest,ypredict)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
CM

array([[1898,  167],
       [ 175, 1417]])

In [ ]:
ytest

array([1, 0, 1, ..., 0, 1, 1])